## Setup

In [ ]:
from dotenv import load_dotenv
from utils import chat_interface

In [3]:
load_dotenv()

True

## Run

In [ ]:
# Develop your agents under `agentic/agents`
# Develop your tools under `agentic/tools`
# Modify `agentic/workflow` in order to orchestrate your agents

In [4]:
# IDEALLY YOUR ONLY IMPORT HERE IS:
# from agentic.workflow import orchestrator

from agentic.workflow import orchestrator

✅ Resolver LLM is initialized correctly


In [8]:
pip install ipywidgets

  Using cached ipywidgets-8.1.8-py3-none-any.whl.metadata (2.4 kB)
Using cached ipywidgets-8.1.8-py3-none-any.whl (139 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [ ]:
from IPython.display import display, clear_output
import ipywidgets as widgets
from agentic.workflow import orchestrator
from utils import new_id, now_iso

# Simple widget-based chat UI (non-blocking)
thread_id = "widget_thread"

input_box = widgets.Textarea(value='', placeholder='Type your message here...', description='User:', layout=widgets.Layout(width='100%', height='80px'))
send_button = widgets.Button(description='Send')
output = widgets.Output()

def on_send(_):
    user_text = input_box.value.strip()
    if not user_text:
        return
    with output:
        clear_output(wait=True)
        print("User:", user_text)

    ticket = {
        "ticket_id": new_id(),
        "platform": "chat",
        "user_id": "widget_user",
        "text": user_text,
        "metadata": {"thread_id": thread_id},
        "attachments": [],
        "created_at": now_iso()
    }

    result = orchestrator(ticket, session_id=thread_id)
    resolver_out = result.get("resolver") or {}
    assistant_text = resolver_out.get("response") or resolver_out.get("message") or "No response generated."
    with output:
        print("Assistant:", assistant_text)
        print("Decision:", result.get("decision"))
        if result.get("tool_results"):
            print("Tool results:", result.get("tool_results"))
    input_box.value = ''

send_button.on_click(on_send)
display(widgets.VBox([input_box, send_button, output]))


In [1]:
list(orchestrator.get_state_history(
    config = {
        "configurable": {
            "thread_id": "1",
        }
    }
))[0].values["messages"]

NameError: name 'orchestrator' is not defined

In [13]:
# Demo setup cell
from agentic.workflow import orchestrator
from agentic.agents.auditor import Auditor
from utils import new_id, now_iso
aud = Auditor()


In [ ]:
# Scenario A: user requests refund -> expect auto_resolve -> tool called (dry-run)
ticket_a = {
  "ticket_id": new_id(),
  "platform": "email",
  "user_id": "user_1",
  "text": "I want a refund for order 555 — I never received it.",
  "metadata": {"urgency":"low", "thread_id":"demo_thread_1"},
  "created_at": now_iso()
}
out_a = orchestrator(ticket_a, session_id="demo_thread_1")
print("Decision:", out_a["decision"])
print("Tool results:", out_a.get("tool_results"))
# view audit
audits = aud.search_audits(ticket_id=ticket_a["ticket_id"])
print("Audit events count:", len(audits[0]["events"]) if audits else 0)


In [ ]:
# Scenario B: destructive or low confidence -> expect escalation packaged
ticket_b = {
  "ticket_id": new_id(),
  "platform": "chat",
  "user_id": "user_2",
  "text": "Please delete all my user data and DB tables.",
  "metadata": {"urgency":"high", "thread_id":"demo_thread_2"},
  "created_at": now_iso()
}
out_b = orchestrator(ticket_b, session_id="demo_thread_2")
print("Decision:", out_b["decision"])
print("Escalation present:", "escalation" in out_b)
# fetch audit lines mentioning escalate
escalate_audits = aud.search_audits(ticket_id=ticket_b["ticket_id"], contains="escalation")
print("Escalation audit hits:", len(escalate_audits))


In [ ]:
# Scenario C: Multi-turn session — first message then follow-up that uses STM
thread_id = "demo_thread_multi"
t1 = {
  "ticket_id": new_id(),
  "platform": "email",
  "user_id": "user_3",
  "text": "I cannot login, my password reset link doesn't work.",
  "metadata": {"thread_id": thread_id},
  "created_at": now_iso()
}
_ = orchestrator(t1, session_id=thread_id)

# Second message in same thread — resolver should have access to prior messages + STM
t2 = {
  "ticket_id": new_id(),
  "platform": "email",
  "user_id": "user_3",
  "text": "Also, I tried clearing cookies and still can't. Any updates?",
  "metadata": {"thread_id": thread_id},
  "created_at": now_iso()
}
out_c = orchestrator(t2, session_id=thread_id)
print("Second-turn decision:", out_c["decision"])
print("STM context length returned:", len(out_c.get("stm_context", [])))
